<a href="https://colab.research.google.com/github/VolhaP87/dsc-tuning-neural-networks-with-normalization-lab-v2-1/blob/master/index_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
from zipfile import ZipFile
data = '/content/Data.zip'
with ZipFile(data, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [3]:
# Load all numeric features
X_train_numeric = pd.read_csv('/content/Data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('/content/Data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('/content/Data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('/content/Data/X_train_cat.csv')
X_val_cat = pd.read_csv('/content/Data/X_val_cat.csv')
X_test_cat = pd.read_csv('/content/Data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('/content/Data/y_train.csv')
y_val = pd.read_csv('/content/Data/y_val.csv')
y_test = pd.read_csv('/content/Data/y_test.csv')

In [4]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [5]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80,69,21453,6,5,1969,1969,0,938,0,...,0,0,0,1,0,0,0,0,1,0
1,60,79,12420,7,5,2001,2001,0,666,0,...,0,0,0,1,0,0,0,0,1,0
2,20,75,9742,8,5,2002,2002,281,0,0,...,0,0,0,1,0,0,0,0,1,0
3,120,39,5389,8,5,1995,1996,0,1180,0,...,0,0,0,1,0,0,0,0,1,0
4,60,85,11003,10,5,2008,2008,160,765,0,...,0,0,0,1,0,0,0,0,1,0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [6]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', 
                                input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 2s 17ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), 
                              columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric),
                            columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric),
                             columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu',
                                        input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [9]:
# Train the model 
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu',
                                  input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 [==============================] - 1s 13ms/step - loss: 0.4839 - mse: 0.4839 - val_loss: 0.1805 - val_mse: 0.1805
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.2564 - mse: 0.2564 - val_loss: 0.1412 - val_mse: 0.1412
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: 0.2065 - mse: 0.2065 - val_loss: 0.1445 - val_mse: 0.1445
Epoch 4/150
33/33 [==============================] - 0s 9ms/step - loss: 0.1738 - mse: 0.1738 - val_loss: 0.1164 - val_mse: 0.1164
Epoch 5/150
33/33 [==============================] - 0s 8ms/step - loss: 0.1591 - mse: 0.1591 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1432 - mse: 0.1432 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1320 - mse: 0.1320 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1219 - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0075 - mse: 0.0075


[0.00751510402187705, 0.00751510402187705]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1356 - mse: 0.1356


[0.13559605181217194, 0.13559605181217194]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28937.103882367253

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu',
                          input_shape=(n_features,),
                          kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 18ms/step - loss: 0.5229 - mse: 0.5229 - val_loss: 0.2694 - val_mse: 0.2694
Epoch 2/150
33/33 [==============================] - 0s 8ms/step - loss: 0.2723 - mse: 0.2723 - val_loss: 0.1945 - val_mse: 0.1945
Epoch 3/150
33/33 [==============================] - 0s 8ms/step - loss: 0.2132 - mse: 0.2132 - val_loss: 0.1742 - val_mse: 0.1742
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1815 - mse: 0.1815 - val_loss: 0.1603 - val_mse: 0.1603
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1641 - mse: 0.1641 - val_loss: 0.1529 - val_mse: 0.1529
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1434 - mse: 0.1434 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1285 - mse: 0.1285 - val_loss: 0.1494 - val_mse: 0.1494
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1160 - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 4ms/step - loss: 0.0063 - mse: 0.0063


[0.00631033955141902, 0.00631033955141902]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1576 - mse: 0.1576


[0.1576029658317566, 0.1576029658317566]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100,
                             activation='relu',
                             input_shape=(n_features,),
                             kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 22ms/step - loss: 0.3910 - mse: 0.3910 - val_loss: 0.1673 - val_mse: 0.1673
Epoch 2/150
33/33 [==============================] - 0s 6ms/step - loss: 0.2203 - mse: 0.2203 - val_loss: 0.1283 - val_mse: 0.1283
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1709 - mse: 0.1709 - val_loss: 0.1127 - val_mse: 0.1127
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1433 - mse: 0.1433 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1271 - mse: 0.1271 - val_loss: 0.1001 - val_mse: 0.1001
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1170 - mse: 0.1170 - val_loss: 0.0946 - val_mse: 0.0946
Epoch 7/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1094 - mse: 0.1094 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 8/150
33/33 [==============================] - 0s 9ms/step - loss: 0.1007 - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0067 - mse: 0.0067


[0.006704471539705992, 0.006704471539705992]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1007 - mse: 0.1007


[0.10065341740846634, 0.10065341740846634]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [21]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', 
                               input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 2s 13ms/step - loss: 0.3665 - mse: 0.3665 - val_loss: 0.1762 - val_mse: 0.1762
Epoch 2/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1755 - mse: 0.1755 - val_loss: 0.1484 - val_mse: 0.1484
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1272 - mse: 0.1272 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1135 - mse: 0.1135 - val_loss: 0.1304 - val_mse: 0.1304
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0884 - mse: 0.0884 - val_loss: 0.2220 - val_mse: 0.2220
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0727 - mse: 0.0727 - val_loss: 0.1324 - val_mse: 0.1324
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0764 - mse: 0.0764 - val_loss: 0.1263 - val_mse: 0.1263
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0458 - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 5ms/step - loss: 0.0047 - mse: 0.0047


[0.004711023531854153, 0.004711023531854153]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 3ms/step - loss: 0.1002 - mse: 0.1002


[0.10024150460958481, 0.10024150460958481]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 12ms/step - loss: 0.4602 - mse: 0.4602 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 2/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1750 - mse: 0.1750 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 3/150
33/33 [==============================] - 0s 7ms/step - loss: 0.1148 - mse: 0.1148 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 4/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0811 - mse: 0.0811 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 5/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0645 - mse: 0.0645 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 6/150
33/33 [==============================] - 0s 8ms/step - loss: 0.0516 - mse: 0.0516 - val_loss: 0.1211 - val_mse: 0.1211
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0424 - mse: 0.0424 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0361 - 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 5.2194e-04 - mse: 5.2194e-04


[0.0005219396553002298, 0.0005219396553002298]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1044 - mse: 0.1044


[0.1044258326292038, 0.1044258326292038]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [27]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 4ms/step - loss: 0.1326 - mse: 0.1326


[0.132609561085701, 0.132609561085701]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [28]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))


28616.658270081218

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.